In [8]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag ,pos_tag_sents
import matplotlib.pyplot as plt
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import vstack
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


In [440]:
embeddings_dict = {}

In [441]:
stop_words=set(stopwords.words("english"))

In [442]:
with open('glove.twitter.27B.50d.txt', 'r' ,encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        emb =np.asarray(values[1:], "float32")
        embeddings_dict[word] = emb

In [443]:
embeddings_dict['is']

array([ 0.18667 ,  0.21368 ,  0.14993 , -0.43155 , -0.33975 ,  0.8475  ,
        0.38977 , -0.13955 , -0.46976 , -0.54834 ,  0.44025 ,  0.069026,
       -6.1569  , -0.5549  ,  0.062301, -0.55152 ,  0.41341 ,  0.62345 ,
       -0.37125 ,  0.3274  ,  0.18377 ,  0.18605 , -0.26863 ,  0.96635 ,
        1.1219  ,  0.7215  ,  0.09091 , -0.64006 , -0.19457 , -0.47882 ,
       -0.32703 ,  0.43049 , -0.24094 , -1.2938  ,  0.77391 , -0.26059 ,
       -0.42837 ,  0.21792 , -0.014106, -0.49876 , -1.117   , -0.89154 ,
       -0.09197 ,  0.28358 , -0.02093 , -0.61461 ,  0.45838 , -0.11498 ,
       -0.24608 , -0.19549 ], dtype=float32)

In [444]:

def remove_link(content):
    '''remove links (using map) and hashtags and mentions'''
    a = content[:content.find('http' or 'https')]
    a = re.sub('#\w+', '', a)
    a = re.sub('@\w+','',a)
    return a

In [445]:
class LemmaTokenizer(object):
    '''lemmatize,tokenize,lower'''
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.token = RegexpTokenizer(r'[a-zA-Z]+')
    def __call__(self, articles):
        return [self.wnl.lemmatize(t).lower() for t in self.token.tokenize(articles) if t.lower() not in stopwords.words('english')]

In [446]:
data = pd.read_csv('SemEval2017-task4-dev.subtask-A.english.INPUT.txt', names = ['id', 'label', 'content'], sep ='\t', index_col=False)
data.head()

,id,label,content
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195045888,negative,If these runway renovations at the airport pre...
3,619974445185302528,neutral,If you could ask an onstage interview question...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...


In [447]:
data.content = data.content.map(remove_link)

In [448]:
# tokenize each content

In [449]:
lt = LemmaTokenizer()
data['token'] = data.content.map(lt)

In [450]:
data.head()

,id,label,content,token
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","[picturehouse, pink, floyd, roger, waters, wal..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,"[order, go, set, watchman, store, website, tue..."
2,619971047195045888,negative,If these runway renovations at the airport pre...,"[runway, renovation, airport, prevent, seeing,..."
3,619974445185302528,neutral,If you could ask an onstage interview question...,"[could, ask, onstage, interview, question, mis..."
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,"[portion, book, sale, harper, lee, go, set, wa..."


In [451]:
def embedder(x):
    '''embed each token with glove (if it exists in DS), also make the vector size 20'''
    t =[]
    for y in x:
        try:
            t.append(embeddings_dict[y])
        except:
            pass
    while len(t)<20:
        t.append([0]*50)
    
    return np.array(t[:20])

In [ ]:
#find number of unique words count


In [610]:
word_index= []
for token in data.token:
    for t in token:
        word_index.append(t)

In [611]:
len(word_index)

224552

In [612]:
len(list(set(word_index)))

20665

In [584]:
data['vector']=data.token.map(embedder)

In [585]:
data.head()

,id,label,content,token,vector,encoded_label
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","[picturehouse, pink, floyd, roger, waters, wal...","[[0.16812999546527863, 0.2084600031375885, 0.3...",1
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...,"[order, go, set, watchman, store, website, tue...","[[-0.5772899985313416, -0.24309000372886658, -...",1
2,619971047195045888,negative,If these runway renovations at the airport pre...,"[runway, renovation, airport, prevent, seeing,...","[[-1.4539999961853027, -0.14899000525474548, 0...",0
3,619974445185302528,neutral,If you could ask an onstage interview question...,"[could, ask, onstage, interview, question, mis...","[[0.01691400073468685, 0.480679988861084, -0.3...",1
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...,"[portion, book, sale, harper, lee, go, set, wa...","[[-0.07840699702501297, -0.4659999907016754, 0...",2


In [586]:
data.iloc[0].vector.shape

(20, 50)

In [587]:
X = []
for i in data.vector:
    X.append(i)

In [588]:
X= np.array(X)
X.shape

(20632, 20, 50)

In [589]:
#test train split and embedding labeld

In [590]:
le = LabelEncoder()
data['encoded_label'] = le.fit_transform(data.label)

In [591]:
batch_size = 128
X_train, X_test, y_train, y_test = train_test_split(
X, data.encoded_label.values, test_size=0.3, random_state=100)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]

X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

In [592]:
X_train.shape

(14442, 20, 50)

In [593]:
# now added LSTM

In [613]:

from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
model=Sequential()
model.add(Input(shape=[20,50]))
model.add(LSTM(64)) # model.add(LSTM(64,return_sequences=True))
model.add(Dense(3, activation='sigmoid'))
print(model.summary())

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_47 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_61 (Dense)             (None, 3)                 195       
Total params: 29,635
Trainable params: 29,635
Non-trainable params: 0
_________________________________________________________________
None


In [614]:
model.compile(loss='sparse_categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'],learning_rate= 0.01)

In [615]:
num_epochs = 6

model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Train on 14314 samples, validate on 128 samples
Epoch 1/6
14314/14314 [==============================] - 11s 768us/sample - loss: 0.9098 - accuracy: 0.5582 - val_loss: 0.7636 - val_accuracy: 0.6719
Epoch 2/6
14314/14314 [==============================] - 7s 467us/sample - loss: 0.8111 - accuracy: 0.6251 - val_loss: 0.6994 - val_accuracy: 0.6875
Epoch 3/6
14314/14314 [==============================] - 6s 441us/sample - loss: 0.7838 - accuracy: 0.6407 - val_loss: 0.6968 - val_accuracy: 0.7188
Epoch 4/6
14314/14314 [==============================] - 6s 440us/sample - loss: 0.7732 - accuracy: 0.6462 - val_loss: 0.6744 - val_accuracy: 0.7109
Epoch 5/6
14314/14314 [==============================] - 6s 443us/sample - loss: 0.7511 - accuracy: 0.6572 - val_loss: 0.6562 - val_accuracy: 0.7109
Epoch 6/6
14314/14314 [==============================] - 6s 446us/sample - loss: 0.7374 - accuracy: 0.6682 - val_loss: 0.6372 - val_accuracy: 0.7031
Test accuracy: 0.64442647


In [465]:
#word2vec with gensim

In [466]:
import gensim 
from gensim.models import Word2Vec 

In [481]:
main_token = []

In [482]:
for t in data.token:
    main_token.append(t.)

In [484]:
type(main_token)

list

In [568]:
model2 = gensim.models.Word2Vec(main_token, min_count = 1, size = 100, window = 10, sg = 1) 

In [570]:
def w2v_embedder(x):
    '''embed each token with glove (if it exists in DS), also make the vector size 20'''
    t =[]
    for y in x:
        try:
            t.append(model2[y])
        except:
            pass
    while len(t)<20:
        t.append([0]*100)
    
    return np.array(t[:20])

In [571]:
data['vector']=data.token.map(w2v_embedder)

C:\Users\98914\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [572]:
data.iloc[0].vector.shape

(20, 100)